In [1]:
prior = 0.041

def compute_prior_away(score, prior):

    if score < prior: score = prior
    score_no_prior = (score - prior) / (1 - prior)

    return score_no_prior


In [2]:
line = '9606.ENSP00000000233 9606.ENSP00000314615 0 0 0 0 0 61 63 0 262 900 0 473 317 971'

l = line.split()

(protein1, protein2,
 neighborhood, neighborhood_transferred,
 fusion, cooccurrence,
 homology,
 coexpression, coexpression_transferred,
 experiments, experiments_transferred,
 database, database_transferred,
 textmining, textmining_transferred,
 initial_combined) = l

In [3]:
neighborhood = float(neighborhood) / 1000
neighborhood_transferred = float(neighborhood_transferred) / 1000
fusion = float(fusion) / 1000
cooccurrence =  float(cooccurrence) / 1000
homology = float(homology) / 1000
coexpression = float(coexpression) / 1000
coexpression_transferred = float(coexpression_transferred) / 1000
experiments = float(experiments) / 1000
experiments_transferred = float(experiments_transferred) / 1000
database = float(database) / 1000
database_transferred = float(database_transferred) / 1000
textmining = float(textmining) / 1000
textmining_transferred = float(textmining_transferred) / 1000
initial_combined = int(initial_combined)

In [4]:
neighborhood_prior_corrected                 = compute_prior_away (neighborhood, prior)             
neighborhood_transferred_prior_corrected     = compute_prior_away (neighborhood_transferred, prior) 
fusion_prior_corrected                       = compute_prior_away (fusion, prior)             
cooccurrence_prior_corrected                 = compute_prior_away (cooccurrence, prior)           
coexpression_prior_corrected                 = compute_prior_away (coexpression, prior)            
coexpression_transferred_prior_corrected     = compute_prior_away (coexpression_transferred, prior) 
experiments_prior_corrected                  = compute_prior_away (experiments, prior)   
experiments_transferred_prior_corrected      = compute_prior_away (experiments_transferred, prior) 
database_prior_corrected                     = compute_prior_away (database, prior)      
database_transferred_prior_corrected         = compute_prior_away (database_transferred, prior)
textmining_prior_corrected                   = compute_prior_away (textmining, prior)            
textmining_transferred_prior_corrected       = compute_prior_away (textmining_transferred, prior) 

In [17]:
neighborhood_both_prior_corrected = 1.0 - (1.0 - neighborhood_prior_corrected) * (1.0 - neighborhood_transferred_prior_corrected)
coexpression_both_prior_corrected = 1.0 - (1.0 - coexpression_prior_corrected) * (1.0 - coexpression_transferred_prior_corrected)
experiments_both_prior_corrected = 1.0 - (1.0 - experiments_prior_corrected) * (1.0 - experiments_transferred_prior_corrected)
database_both_prior_corrected = 1.0 - (1.0 - database_prior_corrected) * (1.0 - database_transferred_prior_corrected)
textmining_both_prior_corrected = 1.0 - (1.0 - textmining_prior_corrected) * (1.0 - textmining_transferred_prior_corrected)

## now, do the homology correction on cooccurrence and textmining:

cooccurrence_prior_homology_corrected = cooccurrence_prior_corrected * (1.0 - homology)
textmining_both_prior_homology_corrected = textmining_both_prior_corrected * (1.0 - homology)

In [8]:
combined_score_one_minus = (
    (1.0 - neighborhood_both_prior_corrected) *
    (1.0 - fusion_prior_corrected) *
    (1.0 - cooccurrence_prior_homology_corrected) *
    (1.0 - coexpression_both_prior_corrected) *
    (1.0 - experiments_both_prior_corrected) *
    (1.0 - database_both_prior_corrected) *
    (1.0 - textmining_both_prior_homology_corrected) ) 

## and lastly, do the 1 - conversion again, and put back the prior *exactly once*

combined_score = (1.0 - combined_score_one_minus)            ## 1- conversion
combined_score *= (1.0 - prior)                              ## scale down
combined_score += prior

In [10]:
combined_score = int(combined_score * 1000)
combined_score

971

In [13]:
coexpression_both_prior_corrected *= (1.0 - prior) 
coexpression_both_prior_corrected += prior
coexpression_both_prior_corrected = int(coexpression_both_prior_corrected * 1000)
coexpression_both_prior_corrected

82

In [29]:
coexpression = 0.061
coexpression_transferred = 0.063

p = 0.041

c = (p+x*y -y - x)/(p-1)

c

0.08254118873826904

In [30]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import re

global spark

# SparkContext.setSystemProperty('spark.executor.memory', '20g')

spark = (pyspark.sql.SparkSession
    .builder
    .appName("phenodigm_parser")
    .config("spark.executor.memory", '10g')
     .config("spark.driver.bindAddress", "localhost")
    .config("spark.driver.memory", '10g')
    .getOrCreate()
)

#   


print('Spark version: ', spark.version)


Spark version:  3.0.0


In [47]:
(
    spark.read.parquet('/Users/dsuveges/Downloads/literature_20210413_cooccurrences_part-00001-553ab579-cf6b-460b-831e-326370e14b90-c000.snappy.parquet')
    .filter(
        (col('type')=='GP-DS') &
        (col('isMapped') == True) & 
        (col('pmid').isNotNull())
    )
    .select(col('pmid'), col('isMapped'), col('type'))
    .printSchema()
)

root
 |-- pmid: integer (nullable = true)
 |-- isMapped: boolean (nullable = true)
 |-- type: string (nullable = true)

